In [30]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Conv2D, Conv3D, Flatten, MaxPool2D, AveragePooling2D, BatchNormalization
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import CSVLogger
from tensorflow.python.keras import metrics
import matplotlib.pyplot as plt
from utilities.rectified_adam import RAdam
import numpy as np
import sys
from utilities.pythonDB import writeToDB, deleteExistingPrimaryKeyDB
from utilities.data_preprocessors import get_random_data, MetricsAfterEachEpoch
from keras import regularizers
from keras.constraints import unit_norm

In [2]:
global MAX_EPOCHS, MAX_BATCH_SIZE, architecture, label, keywords
MAX_EPOCHS, MAX_BATCH_SIZE = 15, 512
architecture = 'AlexNet'
label, keywords = 'noisy_bl-experiments', 'benchmark_testing'

In [34]:
MAX_TRAINING = 40000
MAX_VALIDATION = 5000
MAX_TESTING = 5000

base = 'noisy_dataset/noisy/cifar_10_{}.npy'
x_train_max, y_train = np.load(base.format('x_train')), np.load(base.format('y_train'))
x_test_max, y_test = np.load(base.format('x_test')), np.load(base.format('y_test'))
# cifar = tf.keras.datasets.cifar10 
# (_, _), (x_test_max, y_test) = cifar.load_data()
y_train_max, y_test_max = to_categorical(y_train, num_classes=10), to_categorical(y_test, num_classes=10)

train_cutoff = int(x_train_max.shape[0] - MAX_VALIDATION*4)

x_train, y_train = get_random_data(x_train_max, y_train_max, 0, train_cutoff, MAX_TRAINING)
x_val, y_val = get_random_data(x_test_max, y_test_max, 0, x_test_max.shape[0], int(MAX_TESTING/2))
x_test, y_test = get_random_data(x_test_max, y_test_max, 0, x_test_max.shape[0], MAX_TESTING)

print ("Number of Training Samples: X={}, Y={}".format(x_train.shape[0], y_train.shape[0]))
print ("Number of Validation Samples: X={}, Y={}".format(x_val.shape[0], y_val.shape[0]))
print ("Number of Test Samples: X={}, Y={}".format(x_test.shape[0], y_test.shape[0]))

Number of Training Samples: X=40000, Y=40000
Number of Validation Samples: X=2500, Y=2500
Number of Test Samples: X=5000, Y=5000


In [44]:
def model_extractor(activation_func, weight_decay=1e-4):
    # Creating an AlexNet Classifier
    model = Sequential()

    #Instantiating Layer 1
    model.add(Conv2D(48, kernel_size=(3, 3), strides=(1, 1), activation=activation_func, padding='valid', 
                    kernel_constraint=unit_norm(), kernel_regularizer=regularizers.l2(weight_decay)))
    model.add(MaxPool2D(pool_size=(2, 2), strides=(2, 2)))

    # #Instantiating Layer 2
    model.add(Conv2D(96, kernel_size=(3, 3), strides=(1, 1), activation=activation_func, padding='same', 
                    kernel_regularizer=regularizers.l2(weight_decay)))
    model.add(MaxPool2D(pool_size=(2, 2), strides=(2, 2)))
    model.add(BatchNormalization())
    # #Instantiating Layer 3
    model.add(Conv2D(192, kernel_size=(3, 3), strides=(1, 1), activation=activation_func, padding='same',
                    kernel_regularizer=regularizers.l2(weight_decay)))
    model.add(BatchNormalization())

    # #Instantiating Layer 4
    model.add(Conv2D(192, kernel_size=(3, 3), strides=(1, 1), activation=activation_func, padding='same', 
                    kernel_regularizer=regularizers.l2(weight_decay)))
    model.add(MaxPool2D(pool_size=(2, 2), strides=(2, 2)))
    model.add(BatchNormalization())

    # #Instantiating Layer 5
    model.add(Conv2D(256, kernel_size=(3, 3), strides=(1, 1), activation=activation_func, padding='same', 
                    kernel_regularizer=regularizers.l2(weight_decay)))
    model.add(MaxPool2D(pool_size=(2, 2), strides=(2, 2)))
    model.add(BatchNormalization())
    model.add(Flatten())

    #Instantiating Layer 6
    model.add(Dense(512, activation=activation_func)) 

    # #Instantiating Layer 8
    model.add(Dense(256, activation=activation_func))

    #Output Layer
    model.add(Dense(10, activation='softmax'))
    return model

In [45]:
def diff_optimizer(curr_optimizer, activation_func, label=None, curr_epochs=MAX_EPOCHS):
    model = model_extractor(activation_func)
    model.compile(loss="categorical_crossentropy", optimizer=curr_optimizer, validation_data=(x_val, y_val),
                  metrics=["accuracy", "mae", "mse"])
    history = model.fit(x_train, y_train, epochs=curr_epochs, batch_size=1024).history
    score = model.evaluate(x_test, y_test, batch_size=MAX_BATCH_SIZE)
    
    #Saving the model
#     if label == 'RAdam':
#         model.save("../../saved_models_noisy/{}_{}_{}.hdf5".format('RAdam', activation_func, architecture))
#     else: 
#         model.save("../../saved_models_noisy/{}_{}_{}.hdf5".format(curr_optimizer, activation_func, architecture))
    return score, history

In [46]:
testing_optimizers = ['Adam'] #['RAdam', 'Adam', 'NAdam', 'SGD']
testing_activations = ['tanh']#['tanh', 'relu', 'selu'] #'selu'


for optimizer in testing_optimizers:
    for activation in testing_activations:
        loss, accuracy, mae, mse = {}, {}, {}, {}
        train_loss_dict, train_accuracy_dict, train_mae_dict, train_mse_dict = {}, {}, {}, {}

        if optimizer == 'RAdam':
            score_returned, history = diff_optimizer(RAdam(lr=0.05), activation, label='RAdam')
        else:
            score_returned, history = diff_optimizer(optimizer, activation)


        bg = (score_returned[0], score_returned[1], score_returned[2], score_returned[3], history.get('loss'),\
          history.get('accuracy'), history.get('mae'), history.get('mae'))

        #Delete Existing Primary Keys and then write to DB
        deleteExistingPrimaryKeyDB(optimizer, activation, architecture, label, keywords)
        writeToDB(optimizer, activation, architecture, bg, label, keywords)

print ("Completed Execution for architecture={}".format(architecture))

Train on 40000 samples
Epoch 1/15
40000/40000 [==============================] - 73s 2ms/sample - loss: 1.5837 - acc: 0.4953 - mean_absolute_error: 0.1282 - mean_squared_error: 0.0648
Epoch 2/15
40000/40000 [==============================] - 69s 2ms/sample - loss: 1.1074 - acc: 0.6089 - mean_absolute_error: 0.1019 - mean_squared_error: 0.0512
Epoch 3/15
40000/40000 [==============================] - 70s 2ms/sample - loss: 0.9310 - acc: 0.6858 - mean_absolute_error: 0.0854 - mean_squared_error: 0.0426
Epoch 4/15
40000/40000 [==============================] - 70s 2ms/sample - loss: 0.8014 - acc: 0.7371 - mean_absolute_error: 0.0726 - mean_squared_error: 0.0360
Epoch 5/15
40000/40000 [==============================] - 69s 2ms/sample - loss: 0.6424 - acc: 0.8032 - mean_absolute_error: 0.0573 - mean_squared_error: 0.0277
Epoch 6/15
40000/40000 [==============================] - 70s 2ms/sample - loss: 0.5973 - acc: 0.8205 - mean_absolute_error: 0.0516 - mean_squared_error: 0.0254
Epoch 7/15


In [ ]:
# 47.12% accuracy #Second: 42.2% $Best !! - 47.2

In [ ]:
# Reference :
# Unit Norm - https://machinelearningmastery.com/how-to-reduce-overfitting-in-deep-neural-networks-with-weight-constraints-in-keras/

In [ ]:
# def model_extractor(activation_func, weight_decay=1e-4):
#     # Creating an AlexNet Classifier
#     model = Sequential()

#     #Instantiating Layer 1
#     model.add(Conv2D(48, kernel_size=(3, 3), strides=(1, 1), activation=activation_func, padding='valid', 
#                     kernel_constraint=unit_norm(), kernel_regularizer=regularizers.l2(weight_decay)))
#     model.add(MaxPool2D(pool_size=(2, 2), strides=(2, 2)))

#     # #Instantiating Layer 2
#     model.add(Conv2D(96, kernel_size=(3, 3), strides=(1, 1), activation=activation_func, padding='same', 
#                     kernel_regularizer=regularizers.l2(weight_decay)))
#     model.add(MaxPool2D(pool_size=(3, 3), strides=(2, 2)))
#     model.add(BatchNormalization())
#     model.add(Dropout(0.25))

#     # #Instantiating Layer 3
#     model.add(Conv2D(192, kernel_size=(3, 3), strides=(1, 1), activation=activation_func, padding='same',
#                     kernel_regularizer=regularizers.l2(weight_decay)))
#     model.add(BatchNormalization())

#     # #Instantiating Layer 4
#     model.add(Conv2D(192, kernel_size=(3, 3), strides=(1, 1), activation=activation_func, padding='same', 
#                     kernel_regularizer=regularizers.l2(weight_decay)))
#     model.add(MaxPool2D(pool_size=(2, 2), strides=(2, 2)))
#     model.add(BatchNormalization())
#     model.add(Dropout(0.25))

#     # #Instantiating Layer 5
#     model.add(Conv2D(256, kernel_size=(3, 3), strides=(1, 1), activation=activation_func, padding='same', 
#                     kernel_regularizer=regularizers.l2(weight_decay)))
#     model.add(MaxPool2D(pool_size=(2, 2), strides=(2, 2)))
#     model.add(BatchNormalization())
#     model.add(Flatten())

#     #Instantiating Layer 6
#     model.add(Dense(512, activation=activation_func)) 
#     model.add(Dropout(0.4))

#     # #Instantiating Layer 8
#     model.add(Dense(256, activation=activation_func))
#     model.add(Dropout(0.4))

#     #Output Layer
#     model.add(Dense(10, activation='softmax'))
#     return model